### Learning Objectives
* To compare and contract linked lists with built-in arrays
* To implement the basic operations of linked lists 
* To distinguish singly linked lists from doubly linked lists

### Instructions
Read and study the following sections, run their code examples and solve their challenges. This worksheet has the following challenges:
* [CHALLENGE 01](#ch01)
* [CHALLENGE 02](#ch02)
* [CHALLENGE 03](#ch03)

Run your coding challenges and fix any errors they might have before downloading and submitting your completed worksheet for grading. When done, open the menu **File >> Download as >> HTML (.html)** to download your worksheet in HTML format. **Submit the downloaded *.html* file via Canvas**.

# Linked lists
While built-in arrays are fast, they suffer from three limitations:
- They are fixed in size and their sizes must be known at compile-time.
- They are contiguous which can make it difficult to create very large arrays in fragmented or limited memory.
- It is not efficient to insert an element in the middle of an array because that would require moving some existing elements by one location.

Arrays are made of a set of adjacent cells like the following:

```
    +---+---+---+-----+---+
    | 8 | 5 | 9 | ... | 4 |
    +---+---+---+-----+---+
```
which means that going from one element to the next involves incrementing the current index by one.

Linked lists on the other hand have their elements scattered all over as you see below.


```
                   +---+                                       +---+
                   | 5 |                                       | 4 |
                   +---+       +---+                           +---+
                               | 9 |                    
                               +---+                      
        +---+                                    +-----+    
        | 8 |                                    | ... |
        +---+                                    +-----+
```

For these scattered elements to be made into a list, we have link them together and indicate for each one of them what and where the next element is. We also need to mark where the beginning and end of this list are.

We call the data structure that maintains all these links and marks a **linked list**.Such data structure gives us something like this:

```
                   +---+                                       +---+
               +-->| 5 |---+                               +-->| 4 |<--back
               |   +---+   |    +---+                      |   +---+
               |           +--->| 9 |-----+                |
               |                +---+     |                |
        +---+  |                          |     +-----+    |
front-->| 8 |--+                          +---->| ... |----+
        +---+                                   +-----+
```

with `front` marking the start or the head of the list, `back` the end or the tail of it, and arrows emitting from an element and pointing to the next. But how do we point from one element to another? Well, that is what pointers are for.

Having such a data structure solves all of these array issues above. Its elements, henceforth called **nodes**, are linked to one another using pointers. These nodes don't have to be contiguous; they can be created one by one at runtime and grow to any list size we want. Finally, this list of linked nodes allows for inserting a node in the middle efficiently; we just need to adjust a few pointers.

The linked list example above is what we call a **singly linked list** in which every node has a single pointer pointing to the next element. This list allows us to iterate over its nodes starting with `front` until we reach `back`. It does not however allow for going the opposite direction: `back` to `front`.

Linked lists where each node has two pointers one to the node before it and the other to the node after it are called **doubly linked lists**. Such lists allow for iterating over nodes from `back` to `front`.

Let's see what implementing a singly linked list looks like. Double linked lists are implemented with a few additions in the same way.

## The `node` data structure
First we must recognize the fact that linked lists are containers of data and that they need to support data of any type. That means we have to use C++ templates to make our implementation generic, starting with the following `Node` data structure.

In [ ]:
template<typename T>
struct Node {
  T info;
  Node<T>* next;

  Node(T info): info(info), next(nullptr){}
  Node(T info, Node<T>* ptr): info(info), next(ptr){}
};

Here a node consists of two pieces: `info` for storing the actual value of the node and `next` which is a pointer to the next node in the list. To make creating these nodes easier, notice the two constructors we have for this structure.

We now can use this `Node` structure to create the linked list class. 

## The `List` ADT

In [ ]:
#include <iostream>
#include <initializer_list>

template<typename T>
class List {
  unsigned size = 0u;
  Node<T>* front = nullptr;   
  Node<T>* back = nullptr;
  void deleteAll();
public:
  List();
  List(const std::initializer_list<T>& lst);
  List(const List<T>& lst);
  List<T>& operator=(const List<T>& lst);
    
  List<T>& add_back(T info);
  List<T>& add_front(T info);
    
  bool remove_back();
  bool remove_front();
  bool remove(T info);
  bool find(T info);
    
  friend std::ostream& operator<<(std::ostream& out, const List<T>& lst){
    auto current = lst.front;
    out << "[" << lst.size << "]: ";
    while(current){
      out << current->info << " ";
      current = current->next;
    }

    return out;
  }
    
  unsigned getSize() const { return size; }
    
  ~List();
};

As the class above indicates, a singly linked list has two pointers `front` and `back` marking the first and last nodes in the list. It also maintains a `size` field that quickly tells how many nodes in the list. These pointers allow us to easily iterate over all the nodes of the list using a simple `while` loop like this:

```c++
auto current = front;
while(current){ // current will be false when nullptr at the end of list is encountered
  // TODO: whatever you want to do here
    
  current = current->next;
}
```
This loop pattern is extremely important and you will see it over and over in the implementation of this class. 

## Supporting the creation and deletion of linked lists
Here is the first constructor.

In [ ]:
template<typename T> List<T>::List(){} // empty constructor

No need to initialize `size`, `front`, and `back` since they are already initialized. Now we move to the constructor with the initializer list which allows us to create a list and initialize it like this:
```c++
List<double> lst {2.5, 7.0, 9.5, 17.75};
```
Here is how this constructor is implemented:

In [ ]:
template<typename T> List<T>::List(const std::initializer_list<T>& lst){// initializer-list constructor
  for(auto it = lst.begin(); it != lst.end(); ++it){
    add_back(*it);
  }
}

This constructor takes the given comma-separated values one at a time and call the `add_back` which we will implement later to add them to the back of the list. We now move to the copy constructor which allows us to take an existing linked list and create a copy of it. For example

```c++
List<double> lst {2.5, 7.0, 9.5, 17.75};
List<double> copy = lst;
```

In [ ]:
template<typename T> List<T>::List(const List<T>& lst){ // copy constructor
  auto current = lst.front;
  while(current){
    add_back(current->info);
    current = current->next;
  }
}

The **rule of three** states that if a class defines one (or more) of the following it should probably explicitly define all three:
- destructor
- copy constructor
- copy assignment operator

Let's do that starting with the copy assignment operator.

In [ ]:
template<typename T> 
List<T>& List<T>::operator=(const List<T>& lst){ // copy assignment operator
  deleteAll();
  auto current = lst.front;
  while(current){
    add_back(current->info);
    current = current->next;
  }

  return *this;
}

followed by the destructor

In [ ]:
template<typename T> List<T>::~List(){ deleteAll(); } // destructor

Both the copy assignment operator and the destructor depend on the private member function `deleteAll` which is in charge of cleaning after deleted nodes and ensuring that memory leak does not happen. Here is what this function looks like:

In [ ]:
template<typename T>
void List<T>::deleteAll(){
  auto current = front;
  while(current){
    auto tmp = current;
    current = current->next;
    delete tmp;
  }

  front = nullptr;
  back = nullptr;
  size = 0u;
}

Notice how this function iterates over the existing nodes of the list and deletes them one by one. It then sets the `front` and `back` pointers to `nullptr` and `size` to zero indicating an empty list.

We now turn to adding and removing nodes.

## Adding nodes to linked lists
There are different ways to add nodes to a linked list. Nodes can be added to the back of the list using the `add_back` function or at the front using the `add_front` function.

The `add_back` function must check if the list has a back node and, if so, makes the newly created node the new back. If not, the list is empty and the newly created node will become both the front and back of the list. It finally increments the size of the list by one.

In [ ]:
template<typename T>
List<T>& List<T>::add_back(T info){
  auto node = new Node<T>(info);
  if(back){
    back->next = node;
    back = node;
  } else { // empty list
    front = node;
    back = node;
  }

  size++;
  return *this;
}

Similarly `add_front` needs to make the newly created node the new front of the list and increment the size of the list by one.

In [ ]:
template<typename T>
List<T>& List<T>::add_front(T info){
  front = new Node<T>(info, front);
  if(!back){
    back = front;
  }

  size++;

  return *this;
}

## Removing nodes from linked lists

There are many ways in which nodes can be removed from a linked list. We can remove the node at the back of the list using the `remove_back` function or remove the node at the front of the list using the `remove_front` function. These functions need to deal with situations where the list is empty, the list has only one node, and the list has more than one node. Both functions return `false` if nothing is removed and `true` otherwise. And if a node is removed, the size is decremented by one.

Here is the implementation of the `remove_back` function.

In [ ]:
template<typename T>
bool List<T>::remove_back(){
  if(front == back){
    if(!back){
      return false;
    }

    delete back;
    front = nullptr;
    back = nullptr;
  } else {
    auto pred = front;
    while(pred->next != back){
      pred = pred->next;
    }

    pred->next = nullptr;
    delete back;
    back = pred;
  }
  size--;
  return true;
}

Similarly `remove_front` needs to remove the front node and make its next node (if any) the new front.

In [ ]:
template<typename T>
bool List<T>::remove_front(){
  if(front == back){
    if(front == nullptr){
      return false;
    }

    delete front;
    front = nullptr;
    back = nullptr;
  } else {
    auto tmp = front;
    front = front->next;

    delete tmp;
  }
  size--;
  return true;
}

Given a value, we can also remove the node that has that value from anywhere in the list. This involves searching for that node first. Here is what the `remove` function looks like.

In [ ]:
template<typename T>
bool List<T>::remove(T info){
  if(front == nullptr){
    return false;
  }else if(front->info == info){
    auto tmp = front;
    front = front->next;
    delete tmp;
    size--;
    return true;
  } else {
    auto pred = front;
    auto current = front->next;
    while(current){
      if(current->info == info){
        pred->next = current->next;
        if(!pred->next){
          back = pred;
        }
        delete current;
        size--;
        return true;
      }
      pred = current;
      current = current->next;
    }

    return false;
  }
}

## Searching for a node in a linked list
Our last operation involves being able, given a value, to seach if a node within the list has that value. Here is the `find` function for that.

In [ ]:
template<typename T>
bool List<T>::find(T info){
  auto current = front;
  while(current){
    if(current->info == info){
      return true;
    }

    current = current->next;
  }

  return false;
}

This function returns `true` if the searched-for value is found and `false` otherwise.

### <a id="ch01">CHALLENGE 01</a>
Go back to the code cell where the template class `List` is defined and add the following public member function to it.

```c++
bool add_before(T info, T before);
```
You'll need to rerun all the code cells after that.

This function should create a new node with the given `info` and add it to the list right before the node containing the given `before` value. It then increments the size by one and returns `true`. If no node with the given `before` value is found, then no node is added to the list and `false` is returned.

Implement this function in the code cell below. Run your code and make sure it compiles without errors and warnings.

In [ ]:
// TODO: Implement add_before here

### <a id="ch02">CHALLENGE 02</a>
In the code cell below, create three linked list objects one using each constructor and call all the member functions on each object before printing them out.

In [ ]:
#include <iostream>
using namespace std;

//TODO: Rest of your code goes here

### <a id="ch03">CHALLENGE 03</a>

Consider a linked list with $n$ nodes. Answer the following running-time questions using the Big-O notation as a function of $n$.

**Q1**. What is the running time of adding a node right before the back of the list using your `add_before` function. 

**Q2**. What is the running time of trying to find a value that does not exist in a linked list?

**Q3**. What is the running time of using the copy constructor to create a linked list that is a copy of an existing list of $n$ nodes.